## Embedding API - ZhipuAI

响应示例：

```json
{
    "model": "embedding-3",
    "data": [
        {
            "embedding": [
                -0.02675454691052437,
                0.019060475751757622,
                ...... 
                -0.005519774276763201,
                0.014949671924114227
            ],
            "index": 0,
            "object": "embedding"
        },
        ...
        {
            "embedding": [
                -0.02675454691052437,
                0.019060475751757622,
                ...... 
                -0.005519774276763201,
                0.014949671924114227
            ],
            "index": 2,
            "object": "embedding"
        }
    ],
    "object": "list",
    "usage": {
        "completion_tokens": 0,
        "prompt_tokens": 100,
        "total_tokens": 100
    }
}
```

In [33]:
from zhipuai import ZhipuAI
import os
API_KEY = 'fc4c27cb51ee4722a642b76909d01464.czerMWREZH1xkavS'

def zhipu_embedding(text: str):
    client = ZhipuAI(api_key=API_KEY)
    response = client.embeddings.create(
        model="embedding-2",
        input=text,
    )
    return response

text = '要生成 embedding 的输入文本，字符串形式。'
response = zhipu_embedding(text=text)

In [11]:
response.data[0].embedding[:10]

[0.017893229,
 0.064432174,
 -0.009351327,
 0.027082685,
 0.0040648775,
 -0.05599671,
 -0.042226028,
 -0.030019397,
 -0.01632937,
 0.067769825]

## Data Processing

### 1. 源文档提取

#### PDF

In [3]:
from langchain.document_loaders.pdf import PyMuPDFLoader

# 创建一个 PyMuPDFLoader Class 实例，输入为待加载的 pdf 文档路径
loader = PyMuPDFLoader("knowledge_db/LLM-TAP-v2.pdf")
# 调用 PyMuPDFLoader Class 的函数 load 对 pdf 文件进行加载
pdf_pages = loader.load()

In [4]:
print(f"载入后的变量类型为：{type(pdf_pages)}，",  f"该 PDF 一共包含 {len(pdf_pages)} 页")

载入后的变量类型为：<class 'list'>， 该 PDF 一共包含 533 页


In [5]:
pdf_page = pdf_pages[10]
print(f"每一个元素的类型：{type(pdf_page)}.", 
    f"该文档的描述性数据：{pdf_page.metadata}", 
    f"查看该文档的内容:\n{pdf_page.page_content}", 
    sep="\n------\n")

每一个元素的类型：<class 'langchain_core.documents.base.Document'>.
------
该文档的描述性数据：{'producer': 'xdvipdfmx (20240305)', 'creator': 'LaTeX with hyperref', 'creationdate': '2025-03-05T03:53:41+00:00', 'source': 'knowledge_db/LLM-TAP-v2.pdf', 'file_path': 'knowledge_db/LLM-TAP-v2.pdf', 'total_pages': 533, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2025-03-16T16:49:12+08:00', 'trapped': '', 'modDate': "D:20250316164912+08'00'", 'creationDate': 'D:20250305035341Z', 'page': 10}
------
查看该文档的内容:
1. 绪论
大语言模型是一种由包含数百亿个及以上参数的深度神经网络构建的语言模型，通常使用自
监督学习方法通过大量无标注文本进行训练。2018 年以来，Google、OpenAI、Meta、百度、华为
等公司和研究机构相继发布了BERT[1]、GPT[2] 等多种模型，这些模型在几乎所有自然语言处理任
务中都表现出色。2019 年，大语言模型呈现爆发式的增长，特别是2022 年11 月ChatGPT（Chat
Generative Pre-trained Transformer）的发布，引起了全世界的广泛关注。用户可以使用自然语言与
系统交互，实现问答、分类、摘要、翻译、聊天等从理解到生成的各种任务。大语言模型展现出
了强大的对世界知识的掌握和对语言的理解能力。
本章主要介绍大语言模型的基本概念、发展历程和构建流程。
1.1 大语言模型的基本概念
使用语言是人类与其他动物最重要的区别之一，而人类的多种智能也与此密切相关，逻辑思维
以语言的形式表达，大量的知识也以文字的形式记录和传播。如今，互联网上已经拥有数万亿个网页

#### Markdown

In [15]:
from langchain.document_loaders.markdown import UnstructuredMarkdownLoader

file_path = "knowledge_db/transformers_README.md"

loader = UnstructuredMarkdownLoader(file_path)
md_pages = loader.load()

In [16]:
print(f"载入后的变量类型为：{type(md_pages)}，",  f"该 Markdown 一共包含 {len(md_pages)} 页")

载入后的变量类型为：<class 'list'>， 该 Markdown 一共包含 1 页


In [17]:
md_page = md_pages[0]
print(f"每一个元素的类型：{type(md_page)}.", 
    f"该文档的描述性数据：{md_page.metadata}", 
    f"查看该文档的内容:\n{md_page.page_content[0:][:200]}", 
    sep="\n------\n")

每一个元素的类型：<class 'langchain_core.documents.base.Document'>.
------
该文档的描述性数据：{'source': 'knowledge_db/transformers_README.md'}
------
查看该文档的内容:
Generating the documentation

To generate the documentation, you first have to build it. Several packages are necessary to build the doc, 
you can install them with the following command, at the root 


### 3. 数据清洗

In [18]:
import re
pattern = re.compile(r'[^\u4e00-\u9fff](\n)[^\u4e00-\u9fff]', re.DOTALL)
pdf_page.page_content = re.sub(pattern, lambda match: match.group(0).replace('\n', ''), pdf_page.page_content)
pdf_page.page_content = pdf_page.page_content.replace('•', '')
pdf_page.page_content = pdf_page.page_content.replace(' ', '')

print(pdf_page.page_content)

1.绪论
大语言模型是一种由包含数百亿个及以上参数的深度神经网络构建的语言模型，通常使用自
监督学习方法通过大量无标注文本进行训练。2018年以来，Google、OpenAI、Meta、百度、华为
等公司和研究机构相继发布了BERT[1]、GPT[2]等多种模型，这些模型在几乎所有自然语言处理任
务中都表现出色。2019年，大语言模型呈现爆发式的增长，特别是2022年11月ChatGPT（ChatGenerativePre-trainedTransformer）的发布，引起了全世界的广泛关注。用户可以使用自然语言与
系统交互，实现问答、分类、摘要、翻译、聊天等从理解到生成的各种任务。大语言模型展现出
了强大的对世界知识的掌握和对语言的理解能力。
本章主要介绍大语言模型的基本概念、发展历程和构建流程。1.1大语言模型的基本概念
使用语言是人类与其他动物最重要的区别之一，而人类的多种智能也与此密切相关，逻辑思维
以语言的形式表达，大量的知识也以文字的形式记录和传播。如今，互联网上已经拥有数万亿个网页
的资源，其中大部分信息都是用自然语言描述的。因此，如果人工智能算法想要获取知识，就必须懂
得如何理解人类所使用的不太精确、可能有歧义甚至有些混乱的语言。语言模型（LanguageModel，LM）的目标就是对自然语言的概率分布建模。词汇表V上的语言模型，由函数P(w1w2···wm)表
示，可以形式化地构建为词序列w1w2···wm的概率分布，表示词序列w1w2···wm作为一个句子
出现的可能性的大小。由于联合概率P(w1w2···wm)的参数量巨大，因此直接计算P(w1w2···wm)
非常困难[3]。《现代汉语词典》（第7版）包含约7万词，句子长度按照20个词计算，语言模型的
参数量达到7.9792×1096的天文数字。在中文的书面语中，超过100个词的句子并不罕见，如果
要将所有可能性都纳入考虑，则语言模型的复杂度会进一步增加，以目前的计算手段无法进行存
储和运算。
为了减小P(w1w2···wm)模型的参数空间，可以利用句子序列（通常是从左至右）的生成过
程将其进行分解，使用链式法则可以得到


In [19]:
md_page.page_content = md_page.page_content.replace('\n\n', '\n')
print(md_page.page_content)

Generating the documentation
To generate the documentation, you first have to build it. Several packages are necessary to build the doc, 
you can install them with the following command, at the root of the code repository:
bash
pip install -e ".[docs]"
Then you need to install our special tool that builds the documentation:
bash
pip install git+https://github.com/huggingface/doc-builder
NOTE
You only need to generate the documentation to inspect it locally (if you're planning changes and want to
check how they look before committing for instance). You don't have to commit the built documentation.
Building the documentation
Once you have setup the doc-builder and additional packages, you can generate the documentation by 
typing the following command:
bash
doc-builder build transformers docs/source/en/ --build_dir ~/tmp/test-build
You can adapt the --build_dir to set any temporary folder that you prefer. This command will create it and generate
the MDX files that will be rendered as the

### 4. 文档分割

In [20]:
''' 
* RecursiveCharacterTextSplitter 递归字符文本分割
RecursiveCharacterTextSplitter 将按不同的字符递归地分割(按照这个优先级["\n\n", "\n", " ", ""])，
    这样就能尽量把所有和语义相关的内容尽可能长时间地保留在同一位置
RecursiveCharacterTextSplitter需要关注的是4个参数：

* separators - 分隔符字符串数组
* chunk_size - 每个文档的字符数量限制
* chunk_overlap - 两份文档重叠区域的长度
* length_function - 长度计算函数
'''
#导入文本分割器
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 知识库中单段文本长度
CHUNK_SIZE = 500

# 知识库中相邻文本重合长度
OVERLAP_SIZE = 50
# 使用递归字符文本分割器
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=OVERLAP_SIZE
)
text_splitter.split_text(pdf_page.page_content[0:1000])

['1.绪论\n大语言模型是一种由包含数百亿个及以上参数的深度神经网络构建的语言模型，通常使用自\n监督学习方法通过大量无标注文本进行训练。2018年以来，Google、OpenAI、Meta、百度、华为\n等公司和研究机构相继发布了BERT[1]、GPT[2]等多种模型，这些模型在几乎所有自然语言处理任\n务中都表现出色。2019年，大语言模型呈现爆发式的增长，特别是2022年11月ChatGPT（ChatGenerativePre-trainedTransformer）的发布，引起了全世界的广泛关注。用户可以使用自然语言与\n系统交互，实现问答、分类、摘要、翻译、聊天等从理解到生成的各种任务。大语言模型展现出\n了强大的对世界知识的掌握和对语言的理解能力。\n本章主要介绍大语言模型的基本概念、发展历程和构建流程。1.1大语言模型的基本概念\n使用语言是人类与其他动物最重要的区别之一，而人类的多种智能也与此密切相关，逻辑思维\n以语言的形式表达，大量的知识也以文字的形式记录和传播。如今，互联网上已经拥有数万亿个网页\n的资源，其中大部分信息都是用自然语言描述的。因此，如果人工智能算法想要获取知识，就必须懂',
 '的资源，其中大部分信息都是用自然语言描述的。因此，如果人工智能算法想要获取知识，就必须懂\n得如何理解人类所使用的不太精确、可能有歧义甚至有些混乱的语言。语言模型（LanguageModel，LM）的目标就是对自然语言的概率分布建模。词汇表V上的语言模型，由函数P(w1w2···wm)表\n示，可以形式化地构建为词序列w1w2···wm的概率分布，表示词序列w1w2···wm作为一个句子\n出现的可能性的大小。由于联合概率P(w1w2···wm)的参数量巨大，因此直接计算P(w1w2···wm)\n非常困难[3]。《现代汉语词典》（第7版）包含约7万词，句子长度按照20个词计算，语言模型的\n参数量达到7.9792×1096的天文数字。在中文的书面语中，超过100个词的句子并不罕见，如果\n要将所有可能性都纳入考虑，则语言模型的复杂度会进一步增加，以目前的计算手段无法进行存\n储和运算。\n为了减小P(w1w2···wm)模型的参数空间，可以利用句子序列（通常是从左至右）的生成过\n程将其进行分解，使用链式法则可以得到']

In [23]:
split_docs = text_splitter.split_documents(pdf_pages)
print(f"切分后的文件数量：{len(split_docs)}")
print(f"切分后的字符数（可以用来大致评估 token 数）：{sum([len(doc.page_content) for doc in split_docs])}")

切分后的文件数量：1605
切分后的字符数（可以用来大致评估 token 数）：647216


## Build up knowledge database

In [27]:
import os

file_paths = []
folder_path = 'knowledge_db'
for root, dirs, files in os.walk(folder_path):
    for file in files:
        file_path = os.path.join(root, file)
        file_paths.append(file_path)
print(file_paths)

['knowledge_db/transformers_README.md', 'knowledge_db/LLM-TAP-v2.pdf', 'knowledge_db/dir1/dir12/file2.md', 'knowledge_db/dir1/dir11/dir111/file1.md']


In [29]:
from langchain.document_loaders.pdf import PyMuPDFLoader
from langchain.document_loaders.markdown import UnstructuredMarkdownLoader

# 遍历文件路径并把实例化的loader存放在loaders里
loaders = []

for file_path in file_paths:

    file_type = file_path.split('.')[-1]
    if file_type == 'pdf':
        loaders.append(PyMuPDFLoader(file_path))
    elif file_type == 'md':
        loaders.append(UnstructuredMarkdownLoader(file_path))
        
# 下载文件并存储到text
texts = []

for loader in loaders: texts.extend(loader.load())

In [30]:
text = texts[1]
print(f"每一个元素的类型：{type(text)}.", 
    f"该文档的描述性数据：{text.metadata}", 
    f"查看该文档的内容:\n{text.page_content[0:]}", 
    sep="\n------\n")

每一个元素的类型：<class 'langchain_core.documents.base.Document'>.
------
该文档的描述性数据：{'producer': 'xdvipdfmx (20240305)', 'creator': 'LaTeX with hyperref', 'creationdate': '2025-03-05T03:53:41+00:00', 'source': 'knowledge_db/LLM-TAP-v2.pdf', 'file_path': 'knowledge_db/LLM-TAP-v2.pdf', 'total_pages': 533, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2025-03-16T16:49:12+08:00', 'trapped': '', 'modDate': "D:20250316164912+08'00'", 'creationDate': 'D:20250305035341Z', 'page': 0}
------
查看该文档的内容:
大语言模型
从理论到实践
（第二版）
张奇桂韬郑锐⻩萱菁著
预览版
2025 年3 月5 日
·


In [40]:
from langchain_community.embeddings import ZhipuAIEmbeddings

embedding = ZhipuAIEmbeddings(api_key=API_KEY)
# 定义持久化路径
persist_directory = 'database/vector_db/chroma'

In [41]:
from langchain.vectorstores.chroma import Chroma

vectordb = Chroma.from_documents(
    documents=split_docs[:20], # 为了速度，只选择前 20 个切分的 doc 进行生成；使用千帆时因QPS限制，建议选择前 5 个doc
    embedding=embedding,
    persist_directory=persist_directory  # 允许我们将persist_directory目录保存到磁盘上
)
vectordb.persist()

/tmp/ipykernel_1355385/2600787329.py:8: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


In [42]:
print(f"向量库中存储的数量：{vectordb._collection.count()}")

向量库中存储的数量：40


相似度检索

$$similarity=cos(A,B)=\frac{A\cdot B}{\parallel A\parallel\parallel B\parallel}=\frac{\sum_1^na_ib_i}{\sqrt{\sum_1^na_i^2}\sqrt{\sum_1^nb_i^2}}$$

In [44]:
question="什么是大语言模型"
sim_docs = vectordb.similarity_search(question,k=3)
print(f"检索到的内容数：{len(sim_docs)}")

for i, sim_doc in enumerate(sim_docs):
    print(f"检索到的第{i}个内容: \n{sim_doc.page_content[:200]}", end="\n--------------\n")


检索到的内容数：3
检索到的第0个内容: 
大语言模型
从理论到实践
（第二版）
张奇桂韬郑锐⻩萱菁著
预览版
2025 年3 月5 日
·
--------------
检索到的第1个内容: 
大语言模型
从理论到实践
（第二版）
张奇桂韬郑锐⻩萱菁著
预览版
2025 年3 月5 日
·
--------------
检索到的第2个内容: 
1.2 大语言模型的发展历程. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .4
1.3 大语言模型的构建流程. . . . . . . . . . . .
--------------


MMR检索(Maximum marginal relevance)

核心思想是在已经选择了一个相关性高的文档之后，再选择一个与已选文档相关性较低但是信息丰富的文档。这样可以在保持相关性的同时，增加内容的多样性，避免过于单一的结果。

In [45]:
mmr_docs = vectordb.max_marginal_relevance_search(question,k=3)
for i, sim_doc in enumerate(mmr_docs):
    print(f"MMR 检索到的第{i}个内容: \n{sim_doc.page_content[:200]}", end="\n--------------\n")

MMR 检索到的第0个内容: 
大语言模型
从理论到实践
（第二版）
张奇桂韬郑锐⻩萱菁著
预览版
2025 年3 月5 日
·
--------------
MMR 检索到的第1个内容: 
2.2.3 预训练语言模型实践. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 32
2.3 大语言模型的结构. . . . . . . . . . . . . . . 
--------------
MMR 检索到的第2个内容: 
2.1.1 嵌入表示层. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 18
2.1.2 注意力层. . . . . . . . . 
--------------
